### Imports

In [4]:
import torch
import torch.nn as nn
import networks
import datasetLoader as datasetLoader
import importlib

importlib.reload(datasetLoader)
importlib.reload(networks)


<module 'networks' from 'c:\\Users\\trist\\Git_repos\\BT-ML-PGESAT\\code\\PGExplainer\\networks.py'>

### Testing PyG

In [2]:
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
print(data)

Data(x=[3, 1], edge_index=[2, 4])


### Parameters

In [5]:
learning_rate_gnn = 0.001        #0.001 on ADAM
epochs_graphgnn = 5000
epochs_nodegnn = 1000


learning_rate_mlp = 0.003        #0.003 on ADAM
coefficientSizeReg = 0.05
entropyReg = 1
epochs_mlp = 30

temperature =  0        #???


## Training Loop GNN

In [27]:
train_loader, val_loader, test_loader = datasetLoader.loadDataset('BA2Motif', 1)               # TODO: adjs matrix is not compatible with GraphConv. Needs to be converted to edge_index (see RE_PGE datasets/utils)

temp = next(iter(train_loader))
gnn = networks.GraphGNN(features = temp.x.shape[1], labels=temp.y.shape[0])


gnn_optimizer = torch.optim.Adam(params = gnn.parameters(), lr = learning_rate_gnn)

loss = nn.BCEWithLogitsLoss()

for epoch in range(0, epochs_nodegnn) :
    print(f'\n------------------ EPOCH {epoch + 1} -------------------')

    gnn.train()
    
    for batch_index, data in enumerate(train_loader):
        
        gnn_optimizer.zero_grad()       # why?

        # real label
        label = data.y

        # get model embeddings (node representations)?
        # predicted label
        out = gnn.forward(data.x, data.edge_index, data.batch)

        # calc cross entropy(???)loss between real label and predicted label
        currLoss = loss(out.reshape(-1), label.float())
        print("Loss: ", currLoss)

        # loss backward
        currLoss.backward()
        
        # torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip_max) WHAT IS THIS AND WHY IS THIS

        # optimizer step
        gnn_optimizer.step()

        # avg loss



#mlp_optimizer = torch.optim.Adam(lr = learning_rate_mlp)


"""for i in enumerate(adjs):
    #out = gnn.forward(feas[i], adjs[i].nonzero().t().contiguous())

for epoch in epochs_graphgnn:
    for graph in adjs:
        # calculate latent variables? MLP?
        for k in # k in monte carlo sampling?!
            # sammple graph
            # pred label on sampled graph

    # compute loss
    # update params with backprop"""

    


------------------ EPOCH 1 -------------------
Loss:  tensor(0.7768, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Loss:  tensor(0.7221, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Loss:  tensor(0.6826, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Loss:  tensor(0.6736, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Loss:  tensor(0.7273, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Loss:  tensor(0.7478, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Loss:  tensor(0.6404, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Loss:  tensor(0.7853, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Loss:  tensor(0.7810, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Loss:  tensor(0.6135, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Loss:  tensor(0.6278, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Loss:  tensor(0.6110, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Loss:  tensor(0.6111, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Loss:  tensor(0.5738, grad_fn

KeyboardInterrupt: 

In [ ]:
print(len(test_loader))

print(next(iter(train_loader)))         # edge_index = "map" for edges, x = features, y = labels
#print(next(iter(train_loader)).x)

print("test_loader contains 100 graphs with following labels:")
for i, curr in enumerate(test_loader):
    print(curr.y)
    print(curr.x.shape[1])
    print(curr.batch)

    out = gnn.forward(curr.x, curr.edge_index, curr.batch)
    print(out.)

100
DataBatch(x=[25, 10], edge_index=[2, 52], y=[1], batch=[25], ptr=[2])
test_loader contains 100 graphs with following labels:
tensor([0])
10
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([[0.2440]], grad_fn=<AddmmBackward0>)
tensor([0])
10
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([[0.2392]], grad_fn=<AddmmBackward0>)
tensor([1])
10
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([[0.2350]], grad_fn=<AddmmBackward0>)
tensor([0])
10
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([[0.2379]], grad_fn=<AddmmBackward0>)
tensor([1])
10
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([[0.2447]], grad_fn=<AddmmBackward0>)
tensor([0])
10
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tens

## Testing

In [34]:
from torch_geometric.datasets import ExplainerDataset
from torch_geometric.datasets.graph_generator import BAGraph
from torch_geometric.datasets.motif_generator import HouseMotif
from torch_geometric.datasets.motif_generator import CycleMotif
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader

dataset1 = ExplainerDataset(
            graph_generator=BAGraph(20, 1),
            motif_generator=HouseMotif(),
            num_motifs=1,
            num_graphs=400,
            transform=T.Constant()      # appends value 1 node feature for every node
        )

dataset2 = ExplainerDataset(
            graph_generator=BAGraph(20, 1),
            motif_generator=CycleMotif(5),
            num_motifs=1,
            num_graphs=400,
            transform=T.Constant()
        )

dataset = torch.utils.data.ConcatDataset([dataset1, dataset2])

print(dataset)
dataset[0].y = torch.tensor([0])

print(dataset[0].y)

train_loader = DataLoader(dataset1, batch_size = 1, shuffle = True)

print(next(iter(train_loader)).y)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2,
        3])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2,
        3])


In [40]:
generator1 = torch.Generator().manual_seed(42)
generator2 = torch.Generator().manual_seed(42)
set1, set2 = torch.utils.data.random_split(range(10), [3, 7])
set3, set4, set5 = torch.utils.data.random_split(range(30), [0.3, 0.3, 0.4])

print("Set 1:")
print(set1[0])
print(set1[1])
print(set1[2])
print("Set 2:")
print(set2[0])
print(set2[1])
print(set2[2])
print(set2[3])
print(set2[4])
print(set2[5])
print(set2[6])

Set 1:
9
3
5
Set 2:
2
6
4
8
7
0
1
